In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import pickle

In [17]:
df_train = pd.read_csv("df_train.csv")

In [18]:
df_train_new = df_train[['Madrid_wind_speed','Bilbao_rain_1h','Valencia_wind_speed','load_shortfall_3h']]

In [19]:
y = df_train_new.load_shortfall_3h
X = df_train_new.drop(['load_shortfall_3h'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [20]:
randomforest =RandomForestRegressor()
randomforest = randomforest.fit(train_X,train_y)

In [21]:
y_pred = randomforest.predict(X)

In [22]:
model_save_path = "randomforest_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(randomforest, file)